# Sieci konwolucyjne (splotowe) - CNN

Dziś spróbujemy użyć innej architektury sieci, aby uzyskać lepsze wyniki w rozpoznawaniu mowy. Na początek zajmiemy się siecią konwolucyjną, zwaną też siecią splotową (CNN, *convolutional neural network*), czyli taką, która dokonuje operacji splotu na danych. Sieci konwolucyjne są szczególnie przydatne w rozwiązywaniu problemów związanych z rozpoznawaniem obrazów i systemami wizyjnymi. Są także stosowane w analizie innych sygnałów, również akustycznych - jeśli mamy dwuwymiarową macierz cech (np. spektrogram, macierz współczynników MFCC), możemy potraktować ją jak obraz.

Zanim przejdziemy do opisu parametrów, które należy zdefiniować podczas implementacji CNN oraz definicji pojęć potrzebnych do zrozumienia kodu, zastanówmy się, czemu wprowadzenie warstw konwolucyjnych tak istotnie wpływa na zdolność sieci do poprawnej klasyfikacji obrazów. Dotychczas wszystkie cechy, które podawaliśmy do sieci (oraz innych klasyfikatorów) musiały być w formie wektorów. Jeżeli wyliczaliśmy cechy 2D, np. spektrogram, to przed procesem uczenia zamienialiśmy go na dane jednowymiarowe w taki sposób, że wartości wyliczone dla kolejnych pasm częstotliwości były „doklejane” na końcu pasma poprzedniego. Takie podejście ma dwie zasadnicze wady:

- tracimy informacje dotyczące rozkładu przestrzennego cech,
- dostajemy zazwyczaj bardzo długi wektor danych wejściowych, a w przypadku sieci neuronowych warstwa wejściowa musi mieć tyle neuronów, ile cech będzie do niej podanych. Załóżmy, że chcemy uczyć sieć na spektrogramie o wymiarach np. 50 ramek x 13 pasm - oznacza to 650 neuronów. Zazwyczaj sieć składa się z większej liczby warstw, w których są kolejne neurony, a dla każdego neuronu muszą być wyznaczone wagi - to wszystko sprawia, że otrzymujemy ogromną liczbę parametrów, które muszą być wyznaczone podczas uczenia sieci. Wyznaczenie ich optymalnych wartości będzie w takim razie bardzo trudne, trening potrwa długo, a ryzyko przeuczenia sieci w związku ze zbyt małą liczbą danych będzie duże.

Stosując warstwy konwolucyjne, przystosowane do pracy z danymi 2D, możemy tych problemów uniknąć. Wykonujemy splot danych wejściowych (lub wyjścia jednej z kolejnych warstw sieci) z **filtrem**, którego współczynniki są aktualizowane w procesie uczenia sieci.

## Splot w pytorchu

Splot może jednak dotyczyć nie tylko danych dwuwymiarowych - w PyTorchu można użyć jednej z trzech klas:
    
- [`nn.Conv1d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html#torch.nn.Conv1d) - przeprowadza splot jednowymiarowy, stosowana jest zazwyczaj na sekwencyjnych danych jednowymiarowych (chociaż można go też zastosować na danych 2D), np. do analizy tekstu lub sygnałów audio. Dane wejściowe powinny mieć wymiary `[batch_size, input_channels, signal_length]`.
- [`nn.Conv2d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d) - przeprowadza splot dwuwymiarowy, stosowana jest głownie w analizie obrazów. Dane wejściowe powinny mieć wymiary `[batch_size, input_channels, input_height, input_width]`.
- [`nn.Conv3d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv3d.html#torch.nn.Conv3d) - przeprowadza splot trójwymiarowy, stosowana jest głownie w analizie wideo lub danych przestrzennych (np. MRI struktury anatomicznej). Dane wejściowe powinny mieć wymiary `[batch_size, input_channels, input_depth, input_height, input_width]`.

Pracujemy na tych samych danych, co tydzień temu, będziemy więc używać splotu dwuwymiarowego: klasy `nn.Conv2d`.

## Importy i wczytanie danych

In [ ]:
# !pip install pytorch-ignite

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from ignite.engine import create_supervised_trainer, create_supervised_evaluator, Events
from ignite.metrics import Loss, Accuracy
from ignite.contrib.handlers import ProgressBar
from ignite.handlers import FastaiLRFinder

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
folder = '../lab8/' #zmodyfikuj ścieżkę odpowiednio do lokalizacji plików
feats = np.load(folder+'melspec_feats.npy')
labels = np.load(folder+'labels.npy')

In [ ]:
feats = np.expand_dims(feats, axis=1) #dodanie jednostkowego wymiaru - "kanałów"
feats = feats.astype(np.float32)

X_train, X_val_test, y_train, y_val_test = train_test_split(feats, labels, random_state=42,
                                                            stratify=labels, train_size=0.8)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, random_state=42,
                                                stratify=y_val_test, train_size=0.5)

In [ ]:
feats.shape

In [ ]:
trainset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
valset = TensorDataset(torch.tensor(X_val), torch.tensor(y_val))
testset = TensorDataset(torch.tensor(X_test), torch.tensor(y_test))

train_loader = DataLoader(trainset, batch_size=256)
val_loader = DataLoader(valset, batch_size=256)
test_loader = DataLoader(testset, batch_size=256)

## Definicja sieci

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=10, stride=1, padding=0)
        self.pooling = nn.MaxPool2d(4)
        self.conv1_drop = nn.Dropout2d(p=0.2)
        self.drop1 = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(880, 100)
        self.fc2 = nn.Linear(100, 35)
        self.relu = nn.ReLU()

    def forward(self, x):
#        print(x.shape)
        x = self.conv1(x)
        x = self.relu(self.pooling(x))
#         print(x.shape)
        x = self.conv1_drop(x)
#         print(x.shape)
        x = x.view(-1, 880) #zmiana kształtu danych - działa podobnie do funkcji np.reshape
                            #musimy zmienić kształt danych, ponieważ na wyjściu warstwy konwolucyjnej mamy dane 4D (batch_size x liczba kanałów x liczba_cech x liczba_ramek), a warstwa gęsta przyjmuje dane 2D (batch_size x łączna_liczba_cech)
#         print(x.shape)
        x = self.fc1(x)
#         print(x.shape)
        x = self.drop1(x)
#         print(x.shape)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1) # możemy skorzystać z torch.nn.functional zamiast definiować warstwę w konstruktorze

Przyjrzyjmy się temu, co dzieje się w powyższym kodzie.

W funkcji `__init__` inicjalizujemy warstwy: konwolucyjną (`conv1`), dropuot dwuwymiarowy (`conv1_drop`), droput jednowymiarowy (`drop1`) oraz dwie warstwy gęste (`fc1`, `fc2`).

Inicjalizując warstwy splotowe (w naszym przypadku `conv1`) należy zdefiniować trzy hiperparametry:

- in_channels - liczba kanałów wejściowych,
- out_channels - liczba kanałów wyjściowych,
- kernel_size - rozmiar jądra splotu.

Liczba kanałów wejściowych odnosi się do kanałów tensora podawanego do sieci. Jeżeli mamy wyekstrahowane dane zapisane w macierzy o wymiarach:

`liczba_sygnałów` $\times$ `liczba_cech` $\times$ `liczba_ramek`,

to zamieniamy je na tensor, który będzie podawany do sieci i ten tensor ma wymiary:

`liczba_sygnałów` $\times$ `liczba_kanałów` $\times$ `liczba_cech` $\times$ `liczba_ramek`.

W przypadku naszych danych `liczba_kanałów = 1`. Gdybyśmy analizowali dane będące obrazami kolorowymi zapisanymi w RGB, to liczba kanałów wynosiłaby 3.

Liczba kanałów wyjściowych jest liczbą całkowitą określającą liczbę kanałów tensora, który uzyskamy na wyjściu warstwy konwolucyjnej, czyli tzw. mapy cech (*feature map*).

## Parametry splotu

Rozmiar jądra splotu określa, z jakiej części danych wejściowych obliczymy splot. Dodatkowo, operacja splotu w sieci neuronowej może być przeprowadzona w dwóch wariantach, które określamy parametrem ***padding***:

- `padding='same'` - liczony jest splot filtra i obrazu uzupełnionego zerami tak, by na wyjściu uzyskać taki sam rozmiar danych, jak na wejściu. Jest to szczególnie istotne w przypadku głębokich sieci, ponieważ pozwala przeprowadzić więcej operacji bez redukcji wymiarowości.
- `padding='valid'` - inaczej zapisywany jako `padding=0`, oznacza brak *paddingu*, a w wyniku splotu rozmiar danych ulega redukcji.

![caption](https://i.stack.imgur.com/O01D7.png)


Kolejnym parametrem, który definiujemy podczas inicjalizacji warstwy konwolucyjnej jest ***stride*** (krok), który oznacza, o ile próbek przesuwany jest filtr. Domyślnie `stride=1` i dla takiego przypadku przeprowadziliśmy powyższe obliczenia. Jest to również jedyna wartość, dla której *padding* może przyjąć wartość 'same'.

Wartość parametru *stride* ma wpływ na to, jaki rozmiar danych uzyskamy po operacji splotu przy braku *paddingu* - im większy *stride*, tym większa redukcja wymiarowości.

![caption](https://i.stack.imgur.com/XD2O4.png)

Rozmiar danych, jaki uzyskamy w wyniku operacji splotu można policzyć ze wzorów:

$W_{out} = [\frac{W_{in}+2*p-k}{s}]+1$

$H_{out} = [\frac{H_{in}+2*p-k}{s}]+1$

gdzie $W_{in}$ i $H_{in}$ to odpowiednio szerokość i wysokość obrazu wejściowego, $p$ to rozmiar *paddingu*, $k$ to rozmiar jądra splotu, $s$ to wielkość kroku *stride*, a $W_{out}$ i $H_{out}$ to odpowiednio szerokość i wysokość obrazu wyjściowego.

**Uwaga:** *stride* i *padding* mogą być różne dla każdego z wymiarów, wtedy podczas inicjalizacji należy podać wartości dla obu wymiarów jako (wymiar_dla_wysokości, wymiar_dla_szerokości), np. `stride=(1,2)`.

Więcej na temat operacji splotu w sieciach neuronowych oraz obliczania liczby neuronów wyjściowych można przeczytać [w tym przewodniku](https://arxiv.org/pdf/1603.07285), a [w tym tutorialu](https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md) są animacje ilustrujące poszczególne parametry.

## Pooling

W architekturze sieci pojawiły się dwie rzeczy, których nie omawialiśmy na poprzednich zajęciach. Pierwszą z nich jest warstwa `nn.MaxPool2d`, która wykonuje tzw. ***pooling***, czyli redukcję wymiarowości danych - tym samym zmniejsza liczbę parametrów sieci (wag w kolejnych warstwach) i ryzyko przeuczenia sieci. *Pooling* polega na tym, że wartości mapy cech zawarte w oknie o wymiarach określonych przez `kernel_size` zostają przetworzone na pojedynczą wartość. W przypadku stosowanego przez nas *max-poolingu* ta wartość jest wartością największą w oknie. Definiujemy dla niej parametr *stride*, który podobnie jak w warstwie konwolucyjnej określa, jakie przesunięcie okien ma być zastosowane podczas *poolingu* - domyślnie `stride=None`.

Więcej parametrów, które można zdefiniować znajdziesz w [dokumentacji klasy nn.MaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html#torch.nn.MaxPool2d).

![caption](https://miro.medium.com/max/700/0*lMlVQWvEoUyvr-nW.png)

*Pooling* może być też wykonany poprzez wyznaczenie innej wartości niż maksymalna, np. wartości średniej (*average-pooling*) - wtedy należy użyć klasy [nn.AvgPool2d](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html#torch.nn.AvgPool2d).

Poniżej znajdują się wymiary danych zwracanych na wyjściach kolejnych warstw (jeżeli chcesz je odtworzyć, musisz odkomentować wszystkie funkcje `print` w kodzie z definicją klasy). Przeanalizuj je - czy zgadzają się z tym, co uzyskasz wykonując obliczenia zgodnie z opisanymi wyżej zależnościami?

```
torch.Size([256, 1, 99, 26])
torch.Size([256, 10, 22, 4])
torch.Size([256, 10, 22, 4])
torch.Size([256, 880])
torch.Size([256, 100])
torch.Size([256, 100])
```

## Funkcje aktywacji

Ostatnią nową funkcją użytą podczas definicji sieci jest `F.relu`, która służy do wprowadzenia nieliniowości do sieci neuronowej funkcją **ReLU** (*rectified linear unit*).

Wyobraźmy sobie sieć neuronową złożoną z tysiąca warstw gęstych, gdzie każdą kolejną oznaczamy jako $f_n$. Funkcja przejścia przez taką sieć wyglądałaby tak:

y = $f_{1000}(f_{999}(f_{998}(f_{997}(f_{996}(....(x)...) + c $

Jest to tylko złożenie funkcji liniowych, równoważne ze stworzeniem jednej warstwy gęstej. Dlatego należy do sieci wprowadzić nieliniowość, aby możliwa była jej nauka oraz składanie większej liczby warstw.

Zanim omówimy funkcję ReLU, zastanówmy się, jakie są inne funkcje aktywacji i dlaczego użycie ReLU w sieciach gęstych i konwolucyjnych jest lepszym wyborem.

### Sigmoid

Funkcję logistyczną (zwaną po prostu sigmoidem) poznaliśmy przy okazji omawiania regresji logistycznej - jest to funkcja S-kształtna, która przyjmuje wartości z przedziału $(0,1)$ i dana jest wzorem:

$sigmoid(x) = \frac{1}{1+e^{-x}}$

Ważna różnica między sigmoidem i softmaxem: jeżeli funkcją aktywacji warstwy jest softmax, to wartości wszystkich wyjść sumują się do 1. W przypadku sigmoidu tak nie jest - każde pojedyncze wyjście będzie zwracać wartość z przedziału $(0,1)$, ale ich suma może być większa od 1.

In [ ]:
x = np.linspace(start=-10, stop=10, num=100)
y = 1/(1 + np.exp(-np.array(x)))

plt.plot(x, y)
plt.xlabel("x")
plt.ylabel("sigmoid(X)")
plt.grid()
plt.show()

### Tangens hiperboliczny

Inną często używaną funkcją (a przy okazji domyślną funkcją aktywacji w warstwach rekurencyjnych) jest tangens hiperboliczny - kształtem przypomina sigmoidę, jednak przyjmuje wartości z przedziału $(-1,1)$ i dany jest wzorem:

$tanh(x) = \frac{e^{x}-e^{-x}}{e^{x}+e^{-x}}$

In [ ]:
y = np.tanh(x)

plt.plot(x, y)
plt.xlabel("x")
plt.ylabel("tanh(x)")
plt.grid()
plt.show()

### Problem zanikającego gradientu

Funkcja sigmoidalna i tangens hiperboliczny mają jedną zasadniczą wadę - w przypadku głębokich sieci neuronowych ich użycie prowadzi do wystąpienia problemu zanikającego gradientu (*vanishing gradient*).

Jak mówiliśmy na poprzednich zajęciach, wagi w sieci neuronowej zazwyczaj dobierane są na drodze analizy gradientu (metoda SGD). Gdy funkcja aktywacji przyjmuje wartości z przedziału $(0,1)$ lub $(-1,1)$, podanie bardzo dużych lub bardzo małych wartości na wejściu (dążących do $+\infty$ lub $-\infty$) spowoduje uzyskanie pochodnej bliskiej zeru. Gradienty uzyskiwane na kolejnych warstwach sieci są przez siebie mnożone - jeżeli pomnożymy kilka liczb z przedziału $(0,1)$, to uzyskamy bardzo małą liczbę. Sprawia to, że wraz ze wzrostem liczby warstw gradient dąży do zera, co powoduje, że optymalizacja wag będzie wtedy bardzo nieefektywna lub wręcz niemożliwa, a tym samym sieć nie będzie w stanie nauczyć się w stopniu nas zadowalającym.

### ReLU

Jedną z metod radzenia sobie z problemem zanikającego gradientu jest zastosowanie odpowiedniej funkcji aktywacji. Do takich funkcji zalicza się właśnie funkcja ReLU, której użyliśmy podczas definicji sieci. Funkcja ReLU przyjmuje następujące wartości:

- 0, jeżeli na wejście podana zostanie wartość ujemna,
- wartość wejściową, jeżeli zostanie podana wartość dodatnia.

In [ ]:
def ReLU(x):
    return max(0.0, x)

y = [ReLU(x) for x in x]
plt.plot(x, y)
plt.xlabel("x")
plt.ylabel("ReLU(x)")
plt.grid()
plt.show()

Funkcja ReLU ma kilka istotnych zalet w porównaniu do funkcji sigmoidalnej oraz tangensa hiperbolicznego:

- prostota i szybkość obliczeniowa - nie wymaga użycia funkcji eksponencjalnej;
- może zwrócić wartość 0, a nie jedynie aproksymację zera - dzięki temu możliwe jest uzyskanie na wyjściu macierzy rzadkiej (czyli takiej, w której większość elementów jest równa 0), co przyspiesza proces uczenia;
- zmniejsza problem zanikających gradientów - wartości nie są ograniczane do niewielkich wartości z konkretnego przedziału.

## Trening sieci

Teraz, gdy wyjaśniliśmy już wszystkie nowe pojęcia, możemy przejść do treningu sieci, korzystając z tego samego kodu, którego użyliśmy na poprzednich zajęciach. Od razu użyjemy LRfindera do znalezienia współczynnika uczenia.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# print(device)
criterion = nn.NLLLoss()
model = ConvNet()
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)

In [ ]:
init_model_state = model.state_dict()
init_opt_state = optimizer.state_dict()

In [ ]:
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, metrics={"acc": Accuracy(), "loss": Loss(nn.NLLLoss())}, device=device)
ProgressBar(persist=True).attach(trainer, output_transform=lambda x: {"batch loss": x})

In [ ]:
lr_finder = FastaiLRFinder()
to_save={'model': model, 'optimizer': optimizer}
with lr_finder.attach(trainer, to_save, diverge_th=1.05, start_lr=1e-8, end_lr=1e-3) as trainer_with_lr_finder:
    #domyślnie start_lr jest taki, jak określony w optimizerze, a end_lr=10
    trainer_with_lr_finder.run(train_loader)

print("Suggested LR", lr_finder.lr_suggestion())

In [ ]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
          .format(trainer.state.epoch, metrics['acc'], metrics['loss']))

lr_finder.apply_suggested_lr(optimizer)
print('Training with suggested lr: ', optimizer.param_groups[0]['lr'])
#trainer.run(trainloader, max_epochs=1000)
trainer.run(train_loader, max_epochs=150)

evaluator.run(test_loader)
print('Test set results:', evaluator.state.metrics)

Wyniki uzyskiwane przez powyższą sieć nie są dobre - użyliśmy bardzo prostej architektury, żeby wyjaśnić zasadę działa poszczególnych elementów, z których składa się sieć oraz pokazać, jak dobierać hiperparametry. Jest ona jednak zbyt prosta, żeby dawać dobre rezultaty dla danych, na których pracujemy.

## Normalizacja wsadu

Do architektury sieci często dodaje się jeszcze jeden element: warstwę wykonującą normalizację wsadową (*batch normalization*).

Jeśli ją dodamy, w trakcie treningu wektor aktywacji warstwy jest poddawany normalizacji podczas każdej iteracji (czyli po przejściu przez warstwę każdego *batcha*) - normalizacja prowadzona jest w oparciu o momenty I i II rzędu wyliczane ze wsadu (czyli wartość średnią $\mu_{batch}$ i wariancję $\sigma_{batch}^{2}$). To sprawia, że każdy neuron w każdej iteracji zwraca na wyjściu dane opisane rozkładem (w przybliżeniu) normalnym, dzięki czemu kolejna warstwa dostaje na wejściu dane znormalizowane. Co więcej, dzięki normalizacji za każdym razem skalujemy dane do tego samego zakresu. To prowadzi z kolei do szybszego uczenia sieci oraz zwiększenia stabilności modelu.

Podczas ewaluacji sieci nie możemy wyznaczyć średniej i odchylenia standardowego w taki sposób, jak podczas treningu. Sieć jest już nauczona, więc nie można dla każdego wsadu wyznaczać nowych wartości. Zamiast tego stosuje się średnią i odchylenie będące estymatorami punktowymi wyznaczonymi na podstawie zbioru wszystkich wartości przyjętych przez $\mu_{batch}$ i $\sigma_{batch}$ podczas treningu (czyli średnią ze średnich).

Jeśli będziemy używać normalizacji po warstwie konwolucyjnej, będzie ona dwuwymiarowa; w funkcji `__init__` trzeba zainicjalizować odpowiednią warstwę:

`self.batch_norm = nn.BatchNorm2d(C)`

gdzie C to liczba kanałów zwracanych przez warstwę konwolucyjną poprzedzającą warstwę normalizacyjną (drugi wymiar tensora).

Następnie używamy tej warstwy w funkcji `forward` po warstwie konwolucyjnej:

```
x = self.conv1_drop(x)
x = self.batch_norm(x)
```

Jeżeli wykonujemy normalizację wsadową po warstwie gęstej, używamy normalizacji jednowymiarowej:

`self.batch_norm = nn.BatchNorm1d(L)`

gdzie L to długość wektora aktywacji kolejnej warstwy (czyli liczba wyjść warstwy poprzedzającej).

Następnie używamy normalizacji w funkcji `forward` po warstwie gęstej:

```
x = self.fc1(x)
x = self.batch_norm(x)
```

# Zadanie

Spróbuj zmienić architekturę sieci tak, by uzyskać większą dokładność klasyfikacji (cel: nie mniej niż 0.9). W tym celu:

- zmień funkcje aktywacji warstw,
- zmień/dodaj/usuń dropout,
- dodaj więcej warstw (konwolucyjnych lub gęstych).